In [ ]:
# REDUCE THE PROBLEM TO 10 ASSESTS
# adterwards insert manualy the A array for both inequality constraints
# make sure each constrint recive only x value 
# run the function. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import optimize 

In [2]:
df = pd.read_csv('data/all_stocks_5yr.csv')

## First exploring: 

In [3]:
print(df.shape)

df.head()

(619040, 7)


,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [4]:
df.describe()

,open,high,low,close,volume
count,619029.000000,619032.000000,619032.000000,619040.000000,6.190400e+05
mean,83.023334,83.778311,82.256096,83.043763,4.321823e+06
std,97.378769,98.207519,96.507421,97.389748,8.693610e+06
min,1.620000,1.690000,1.500000,1.590000,0.000000e+00
25%,40.220000,40.620000,39.830000,40.245000,1.070320e+06
50%,62.590000,63.150000,62.020000,62.620000,2.082094e+06
75%,94.370000,95.180000,93.540000,94.410000,4.284509e+06
max,2044.000000,2067.990000,2035.110000,2049.000000,6.182376e+08


In [5]:
df['Name'].nunique()

505

In [6]:
N = 100

In [7]:
# Choosing the top 100 assets based on mean volume:
list_top_100_assets = df.groupby(['Name'])['volume'].mean().sort_values(ascending=False).head(N).reset_index()['Name'].tolist()


In [8]:
#list_top_100_assets

In [9]:
df_top_100 = df[df['Name'].isin(list_top_100_assets)].reset_index(drop=True)

In [10]:
df_top_100.head()

,date,open,high,low,close,volume,Name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [11]:
# Verifing that the data contains only 100 assests: 
df_top_100['Name'].nunique()

100

In [12]:
# "AAL":{'beta':beta, 'return_stock':return value}

In [13]:
# adding a column that contain the mean per day to detemine the market close per this day. 
df_top_100['market_close']=df_top_100.groupby('date')['close'].transform('mean')

In [14]:
df_top_100

,date,open,high,low,close,volume,Name,market_close
0,2013-02-08,15.07,15.12,14.6300,14.75,8407500,AAL,34.107894
1,2013-02-11,14.89,15.01,14.2600,14.46,8882000,AAL,34.075976
2,2013-02-12,14.45,14.51,14.1000,14.27,8126000,AAL,34.168396
3,2013-02-13,14.30,14.94,14.2500,14.66,10259500,AAL,34.169845
4,2013-02-14,14.94,14.96,13.1600,13.99,31879900,AAL,34.144170
...,...,...,...,...,...,...,...,...
122320,2018-02-01,87.50,89.25,87.3500,89.07,17971012,XOM,62.705800
122321,2018-02-02,85.13,86.01,82.9978,84.53,29822144,XOM,61.241900
122322,2018-02-05,83.28,83.99,78.1300,79.72,30452693,XOM,58.534050
122323,2018-02-06,78.51,80.35,76.9000,78.35,36262761,XOM,59.817900


In [15]:
df_1 = df_top_100[['date','close','Name']]

In [16]:
df_2 = pd.DataFrame()
df_2 = df_top_100[['date','market_close']].drop_duplicates()
df_2['Name'] = 'market'
df_2.rename(columns={'market_close':'close'},inplace=True)

In [17]:
df_all = pd.concat([df_1,df_2])

In [18]:
df_all['Name'].nunique()

101

In [19]:
df_all.rename(columns={'Name':'name'},inplace=True)
df_all = df_all.reset_index(drop=True)
df_all = df_all[['date','name','close']]

In [20]:
data = df_all.pivot(index='date',columns='name',values='close')#.sort_values(by='date', ascending=False)

In [21]:
#data

In [23]:
returns = (data/data.shift())-1
#returns

In [24]:
# calculation for beta:
cov = returns.cov()
var = returns['market'].var()

In [25]:
beta = cov/var

In [26]:
# only last row relevent: 
beta

name,AAL,AAPL,ABBV,ABT,AES,AIG,AMAT,AMD,ARNC,ATVI,...,V,VLO,VZ,WBA,WFC,WMB,WMT,WU,XOM,market
name,,,,,,,,,,,,,,,,,,,,,
AAL,7.431826,1.014209,1.256612,1.183731,0.999178,1.314003,1.847715,1.547542,2.122149,1.429104,...,1.443675,1.647406,0.589157,1.318253,1.331871,1.899674,0.626937,1.212076,0.638297,1.369293
AAPL,1.014209,3.138383,0.659508,0.705832,0.627705,0.814492,1.416043,1.521809,1.299941,1.178252,...,0.894127,0.864660,0.415884,0.764299,0.717059,1.362832,0.437141,0.769475,0.607898,0.880072
ABBV,1.256612,0.659508,4.187366,1.321152,0.851681,0.963635,1.212469,1.089881,1.089251,0.997606,...,0.983831,1.290963,0.575051,1.116817,0.996656,1.774325,0.525932,0.915982,0.812545,1.030664
ABT,1.183731,0.705832,1.321152,2.237585,0.881178,0.969509,1.121065,1.377815,1.167698,1.006621,...,1.058769,1.064817,0.636820,1.027508,1.000974,1.247276,0.631613,0.911345,0.734844,0.940310
AES,0.999178,0.627705,0.851681,0.881178,3.710526,0.918692,1.134641,1.604417,1.691995,1.016396,...,0.898669,1.101919,0.741979,0.889235,0.743012,2.297273,0.560844,0.953205,1.018535,0.931860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WMB,1.899674,1.362832,1.774325,1.247276,2.297273,1.519244,2.031827,2.505013,2.760734,1.577068,...,1.455126,2.452441,0.850176,1.209299,1.541541,12.876775,0.444106,1.636860,1.798275,1.644724
WMT,0.626937,0.437141,0.525932,0.631613,0.560844,0.462623,0.587556,0.500995,0.674705,0.434230,...,0.535029,0.538317,0.495140,0.700006,0.510307,0.444106,1.754636,0.578294,0.395796,0.550448
WU,1.212076,0.769475,0.915982,0.911345,0.953205,1.178912,1.212334,1.655975,1.505606,0.977046,...,1.104757,1.003246,0.615153,0.907147,1.070083,1.636860,0.578294,3.043964,0.780694,0.984163


In [27]:
# Validating the beta using linear regression: 
# x=log_returns['market'].iloc[1:].to_numpy().reshape(-1,1)
# y=log_returns['ABBV'].iloc[1:].to_numpy().reshape(-1,1)

In [28]:
# linalg = LinearRegression()
# linalg.fit(x,y)
# linalg.coef_

In [29]:
#log_returns = np.log(data/data.shift())

In [30]:
df_beta = pd.DataFrame(beta.iloc[-1])

In [31]:
df_beta.reset_index(inplace=True)
df_beta = df_beta.rename(columns={'market':'beta'})

In [32]:
df_beta = df_beta.iloc[:100]

In [33]:
# determine the assets that are high voliatality: 
high_vol = df_beta[df_beta['beta']>1]['name'].tolist()

In [34]:
len(high_vol)

57

In [35]:
# determine the assets that are low voliatality: 
low_vol = df_beta[df_beta['beta']<=1]['name'].tolist()

In [36]:
np.where(df_beta['beta']<=1,1,0)

array([0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1])

In [37]:
len(low_vol)

43

In [38]:
# len(high_vol)
# len(low_vol)

In [39]:
# calculating the mean for each asset: 
df_means = pd.DataFrame(np.mean(returns, axis=0))
df_means.reset_index(inplace=True)
df_means = df_means.rename(columns={0:'mean_return'})
df_means = df_means.iloc[0:100]

In [40]:
df_means.head()

,name,mean_return
0,AAL,0.001245
1,AAPL,0.000786
2,ABBV,0.001050
3,ABT,0.000501
4,AES,0.000082


In [41]:
# mean return:
mean_return = np.mean(returns, axis=0).values[0:100]

In [42]:
# cov return:
cov_returns=cov.values[0:100,0:100]

In [59]:
class Constraints():
    
    def __init__(self, w, df_beta):
        self.w = w
        self.df_beta = df_beta
    
    @staticmethod
    def constraint_eq(self):
        A=np.ones(self.w.shape)
        b=1
        const_val = A@self.w.T-b
        return const_val
    
    def constraint_ineq_upper_bound(self):
        A = [np.where(self.df_beta['beta']<=1,1,0), np.where(self.df_beta['beta']>1,0,1)]
        upper_bound = np.array([0.6,0.4]).T
        contraint_upper = upper_bound-(A@self.w.T)
        return contraint_upper
    
    def constraint_ineq_lower_bound(self):
        A = [np.where(self.df_beta['beta']<=1,1,0), np.where(self.df_beta['beta']>1,0,1)]
        lower_bound = np.array([0.01,0.01]).T
        contraint_lower = (A@self.w.T)-lower_bound
        return contraint_lower
    
    def bound(self):
        return self.df_beta['beta'].apply(lambda x: (0,1) if x<=1 else (0,0.1)).values
        

In [60]:
class ObjectiveFunction():

    def __init__(self, w):
        self.w = w

    def f(self, mean_return, cov_return, gamma, portfolio_size):
        port_var = (self.w@cov_return)@self.w.T
        port_exp_return = np.array(mean_return)@self.w.T
        obj_func = gamma*port_var-(1-gamma)*port_exp_return
        return obj_func

In [61]:
# # validation:
# A=np.ones(w_init.shape)
# b=1
# A@w_init.T-b

# A = [np.where(df_beta['beta']<=1,1,0), np.where(df_beta['beta']>1,0,1)]
# upper_bound = np.array([0.6,0.4]).T
# upper_bound-(A@w_init.T)

# A = [np.where(df_beta['beta']<=1,1,0), np.where(df_beta['beta']>1,0,1)]
# lower_bound = np.array([0.01,0.01]).T
# (A@w_init.T)-lower_bound 

# obj_func.f(mean_return,cov_returns,gamma,portfolio_size)

# # validation:
# gamma =0.3
# port_var = (w_init@cov_returns)@w_init.T
# port_exp_return = np.array(mean_return)@w_init.T
# gamma*port_var-(1-gamma)*port_exp_return

In [62]:
#df_beta['beta_tuple'] = df_beta['beta'].apply(lambda x: (0,1) if x<=1 else (0,0.1))

In [63]:
portfolio_size = df_beta.shape[0]

In [64]:
# since this is the lower bound we will start by generating this weight for each asset:
w_init=np.repeat(0.01, portfolio_size)
# generating the cons class:
constr = Constraints(w_init,df_beta)

In [65]:
cons = ({'type': 'eq', 'fun':constr.constraint_eq},
        {'type':'ineq', 'fun': constr.constraint_ineq_upper_bound},
        {'type':'ineq', 'fun': constr.constraint_ineq_lower_bound})

In [66]:
cons

({'type': 'eq', 'fun': <function __main__.Constraints.constraint_eq(self)>},
 {'type': 'ineq',
  'fun': <bound method Constraints.constraint_ineq_upper_bound of <__main__.Constraints object at 0x7f9899a2fc10>>},
 {'type': 'ineq',
  'fun': <bound method Constraints.constraint_ineq_lower_bound of <__main__.Constraints object at 0x7f9899a2fc10>>})

In [67]:
bnd = constr.bound()

In [68]:
obj_func = ObjectiveFunction(w_init)

In [69]:
#obj_func.f(mean_return,cov_returns,gamma,portfolio_size)

In [70]:
# -----------------------------------------

In [92]:

def bi_function_optimization(mean_return,cov_return, gamma, portfolio_size):
    
    
    def f(w, mean_return, cov_return, gamma, portfolio_size):
        port_var = (w@cov_return)@w.T
        port_exp_return = np.array(mean_return)@w.T
        obj_func = gamma*port_var-(1-gamma)*port_exp_return
        return obj_func
    
    def constraint_eq(w):
        A=np.ones(w.shape)
        b=1
        const_val = A@w.T-b
        return const_val

    def constraint_ineq_upper_bound(df_beta, w):
        A = np.where(df_beta['beta']<=1,1,0) # , np.where(df_beta['beta']>1,0,1)]
        upper_bound = 0.6 #np.array([0.6,0.4]).T
        contraint_upper = upper_bound-(A@w.T)
        return contraint_upper

    def constraint_ineq_lower_bound(df_beta, w):
        A = np.where(df_beta['beta']<=1,1,0)#, np.where(df_beta['beta']>1,0,1)]
        lower_bound = 0.01 #np.array([0.01,0.01]).T
        contraint_lower = (A@w.T)-lower_bound 
        return contraint_lower

    bnd = tuple(df_beta['beta'].apply(lambda x: (0,1) if x<=1 else (0,0.1)).values)

    w=np.repeat(0.01, portfolio_size)
        
    cons = ({'type': 'eq', 'fun':constraint_eq},
        {'type':'ineq', 'fun': constraint_ineq_upper_bound},
        {'type':'ineq', 'fun': constraint_ineq_lower_bound})
    
    
    
    obj_func = optimize.minimize(f, 
                                x0=w,
                                args =(mean_return,cov_return, gamma, portfolio_size),
                                method='SLSQP',
                                bounds=bnd,
                                constraints=cons,
                                tol=10**-3)
    
    print(obj_func)
    return obj_func
    
    

In [93]:
portfolioSize = df_beta.shape[0]

#initialization
xOptimal =[]
minRiskPoint = []
expPortfolioReturnPoint =[]

for points in range(0,60):
    gama = points/60.0
    result = bi_function_optimization(mean_return, cov_returns, gama,portfolioSize)
    xOptimal.append(result.w)

     fun: -0.0005785587385458863
     jac: array([-1.24536122e-03, -7.86196695e-04, -1.05022386e-03, -5.00546084e-04,
       -8.24425151e-05, -4.38579336e-04, -1.16256914e-03, -1.88166254e-03,
       -2.53368744e-04, -1.48056175e-03, -9.01188854e-04, -1.53554442e-03,
       -5.41659821e-04, -1.10269676e-03, -5.61304798e-04, -4.72024323e-04,
       -2.86285052e-04,  6.84405852e-04, -6.53848125e-04, -1.07748172e-04,
       -1.44946331e-04, -6.00653802e-04, -8.32520447e-04,  5.28288489e-04,
       -3.73980009e-04, -8.13023071e-05, -1.23783816e-03, -7.35607755e-04,
       -5.84726484e-04, -5.46619609e-04, -1.17055075e-04, -2.06542849e-04,
        5.27155134e-05, -1.66042167e-03, -2.69637530e-05, -6.36343910e-04,
       -3.17825419e-04,  2.36585009e-04, -7.17664610e-04, -8.11083693e-04,
       -4.28988460e-04, -3.34327138e-04, -7.28390987e-04, -8.96989819e-04,
       -2.24272742e-04, -1.10063685e-03, -2.25947479e-04, -7.06364983e-04,
       -4.81434567e-04, -2.84023721e-04, -7.51915031e-04,

AttributeError: w

In [77]:
gamma =0.3

for i in range(0,2):
    gam = float(i/2)
    obj_func = optimize.minimize(f(w_init,mean_return,cov_returns,gam,portfolio_size), 
                                 x0=w_init, 
                                 #args=(mean_return,cov_returns,gamma,portfolio_size), 
                                 #method='SLSQP',
                                 #bounds=tuple(bnd), #bnd.tolist(), 
                                 constraints=cons_1, 
                                 tol=10**-3)


TypeError: constraint_ineq_upper_bound() missing 1 required positional argument: 'w'